In [2]:
# NumPy, Pandas, Matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# TensorFlow와 Keras
import tensorflow as tf
from keras.models import load_model
import keras


# OpenCV (이미지 처리용)
import cv2

# Jupyter Notebook 환경에서 Matplotlib 인라인 설정
%matplotlib inline

# 확인용 버전 출력
print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")
print(f"OpenCV Version: {cv2.__version__}")

data_path = 'C:/Users/User/Desktop/Python/photofinder/data'

TensorFlow Version: 1.3.0
Keras Version: 2.1.2
Pandas Version: 1.1.5
NumPy Version: 1.19.5
OpenCV Version: 3.4.2


In [7]:
correct_images = np.load(data_path + "/array_ydata.npy")  # 예: (n1, 160, 160, 3)
incorrect_images = np.load(data_path + "/array_ndata.npy")  # 예: (n2, 160, 160, 3)
print(correct_images.shape, incorrect_images.shape)

(8646, 160, 160, 3) (12843, 160, 160, 3)


In [11]:
facenet_model = load_model(data_path + "/facenet_keras.h5")

In [12]:
# 정규화
correct_images = correct_images.astype('float32') / 255.0
incorrect_images = incorrect_images.astype('float32') / 255.0

# 정답 이미지를 FaceNet에 입력하여 임베딩 생성
correct_embeddings = facenet_model.predict(correct_images, verbose=1)
print(f"Correct Embeddings Shape: {correct_embeddings.shape}")  # (n1, 128)

# 오답 이미지를 FaceNet에 입력하여 임베딩 생성
incorrect_embeddings = facenet_model.predict(incorrect_images, verbose=1)
print(f"Incorrect Embeddings Shape: {incorrect_embeddings.shape}")  # (n2, 128)

8646/8646 [==============================] - 178s 21ms/step
Correct Embeddings Shape: (8646, 128)
12843/12843 [==============================] - 271s 21ms/step
Incorrect Embeddings Shape: (12843, 128)


In [16]:
np.save(data_path + '/emb_ydata.npy', correct_embeddings)
np.save(data_path + '/emb_ndata.npy', incorrect_embeddings)

In [12]:
import tensorflow as tf
import numpy as np

# =======[재현]=======

correct_emb = np.load(data_path + '/emb_ydata.npy')
incorrect_emb = np.load(data_path + '/emb_ndata.npy')

print(correct_emb.shape, incorrect_emb.shape)

labels_truth = np.ones((correct_emb.shape[0],), dtype='float32')  # 진실 → 1
labels_false = np.zeros((incorrect_emb.shape[0],), dtype='float32')  # 거짓 → 0

print(labels_truth.shape, labels_false.shape)

X = np.vstack((correct_emb, incorrect_emb))  # 입력 데이터 병합
y = np.hstack((labels_truth, labels_false))        # 레이블 병합

print(X.shape, y.shape)

# TensorFlow 데이터셋 생성
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(buffer_size=len(X), seed=42) # 셔플

# 학습/테스트 데이터 분할 (7:3 비율)
train_size = int(len(X) * 0.7)
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

print(f"Train Dataset: {train_dataset}")
print(f"Test Dataset: {test_dataset}")

(8646, 128) (12843, 128)
(8646,) (12843,)
(21489, 128) (21489,)


AttributeError: module 'tensorflow' has no attribute 'data'

In [11]:
tf.

<module 'tensorflow' from 'C:\\Users\\User\\anaconda3\\envs\\facenet\\lib\\site-packages\\tensorflow\\__init__.py'>

# 테스트에 사용된 부분

In [15]:
# NumPy, Pandas, Matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# TensorFlow와 Keras
import tensorflow as tf
from keras.models import load_model
import keras

data_path = 'C:/Users/User/Desktop/Python/photofinder/data'
facenet_model = load_model(data_path + "/facenet_keras.h5")
tester = np.load("C:\\Users\\User\\Desktop\\Python\\photofinder\\data/test_datas/processed_images.npy")

C:\Users\User\anaconda3\envs\facenet\lib\site-packages\keras\models.py:252: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [17]:
embedding_array = facenet_model.predict(tester)
print(embedding_array.shape)

(12, 128)


In [18]:
np.save("C:\\Users\\User\\Desktop\\Python\\photofinder\\data/test_datas/emb_array.npy", embedding_array)